In [ ]:
import json
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForTokenClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from typing import List, Dict, Tuple
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define input/output paths for Kaggle (เหมือนเดิม)
INPUT_DIR = "/kaggle/input/techstack-ner-dataset-noaugmentate"
OUTPUT_DIR = "/kaggle/working/ner_roberta_results"
TRAIN_DATA_PATH = os.path.join(INPUT_DIR, "train_data.json")
VALIDATE_DATA_PATH = os.path.join(INPUT_DIR, "validate_data.json")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load dataset (เหมือนเดิม)
def load_data(file_path: str) -> List[Dict]:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

train_data = load_data(TRAIN_DATA_PATH)
validate_data = load_data(VALIDATE_DATA_PATH)

# Define label mapping (เหมือนเดิม)
id2label = {
    0: "O",
    1: "B-CLOUDPLATFORM",
    2: "I-CLOUDPLATFORM",
    3: "B-PROGRAMMINGLANG",
    4: "I-PROGRAMMINGLANG",
    5: "B-FRAMEWORK_LIB",
    6: "I-FRAMEWORK_LIB",
    7: "B-WEBFRAMEWORK_TECH",
    8: "I-WEBFRAMEWORK_TECH",
    9: "B-DATABASE",
    10: "I-DATABASE",
    11: "B-EMBEDDEDTECH",
    12: "I-EMBEDDEDTECH"
}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

# Custom Dataset class (เหมือนเดิม)
class NERDataset(Dataset):
    def __init__(self, data: List[Dict], tokenizer, max_length: int = 512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        item = self.data[idx]
        tokens = item["tokens"]
        ner_tags = item["ner_tags"]

        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_offsets_mapping=True
        )
        
        word_ids = encoding.word_ids(batch_index=0)
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        
        aligned_labels = [-100] * self.max_length
        for i, word_idx in enumerate(word_ids):
            if word_idx is None:
                aligned_labels[i] = -100
            else:
                aligned_labels[i] = ner_tags[word_idx]

        encoding["labels"] = torch.tensor(aligned_labels, dtype=torch.long)
        del encoding["offset_mapping"]
        return encoding

# Initialize tokenizer and model (เหมือนเดิม)
tokenizer = RobertaTokenizerFast.from_pretrained("FacebookAI/roberta-large", add_prefix_space=True)
model = RobertaForTokenClassification.from_pretrained(
    "FacebookAI/roberta-large",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Create datasets (เหมือนเดิม)
train_dataset = NERDataset(train_data, tokenizer)
validate_dataset = NERDataset(validate_data, tokenizer)

# Compute class weights (เหมือนเดิม)
all_labels = [label for item in train_data for label in item["ner_tags"] if label != -100]
valid_labels = [label for label in all_labels if label >= 0 and label < num_labels]
unique_labels = np.unique(valid_labels)
print(f"Unique labels in training data: {unique_labels}")

class_weights = compute_class_weight(class_weight="balanced", classes=unique_labels, y=valid_labels)
full_class_weights = np.ones(num_labels)
for i, label in enumerate(unique_labels):
    full_class_weights[label] = class_weights[i]
class_weights = torch.tensor(full_class_weights, dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")
print(f"Class weights: {class_weights}")

# Custom Trainer with weighted loss (เหมือนเดิม)
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Training arguments (เหมือนเดิม)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    save_steps=500,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    num_train_epochs=30,
    weight_decay=0.01,
    logging_dir=os.path.join(OUTPUT_DIR, "logs"),
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,
    report_to=["none"],
)

# Compute metrics (ปรับเพิ่ม Confusion Matrix)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    
    true_labels = [label for sublist in labels for label in sublist if label != -100]
    true_preds = [pred for sublist, label_sublist in zip(predictions, labels) 
                  for pred, label in zip(sublist, label_sublist) if label != -100]
    
    # Metrics for all labels (including "O")
    unique_eval_labels_all = np.unique(np.concatenate([np.unique(true_labels), np.unique(true_preds)]))
    eval_target_names_all = [id2label[i] for i in unique_eval_labels_all]
    report_all = classification_report(
        true_labels,
        true_preds,
        labels=unique_eval_labels_all,
        target_names=eval_target_names_all,
        output_dict=True
    )
    
    # Metrics for entities only (excluding "O")
    entity_true = [label for label in true_labels if label != 0]
    entity_preds = [pred for pred, true in zip(true_preds, true_labels) if true != 0]
    if entity_true:
        unique_eval_labels_entity = np.unique(np.concatenate([np.unique(entity_true), np.unique(entity_preds)]))
        eval_target_names_entity = [id2label[i] for i in unique_eval_labels_entity]
        report_entity = classification_report(
            entity_true,
            entity_preds,
            labels=unique_eval_labels_entity,
            target_names=eval_target_names_entity,
            output_dict=True
        )
        entity_f1 = report_entity["weighted avg"]["f1-score"]
    else:
        entity_f1 = 0.0
    
    # Compute Confusion Matrix
    cm = confusion_matrix(true_labels, true_preds, labels=unique_eval_labels_all)
    cm_df = pd.DataFrame(cm, index=eval_target_names_all, columns=eval_target_names_all)
    print("\n=== Confusion Matrix (Evaluation) ===")
    print(cm_df.to_string())
    
    # Visualize Confusion Matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title("Confusion Matrix (Evaluation Dataset)")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    plt.tight_layout()
    cm_plot_path = os.path.join(OUTPUT_DIR, "confusion_matrix_eval.png")
    plt.savefig(cm_plot_path)
    print(f"Confusion Matrix saved to {cm_plot_path}")
    plt.close()

    return {
        "precision": report_all["weighted avg"]["precision"],
        "recall": report_all["weighted avg"]["recall"],
        "f1": report_all["weighted avg"]["f1-score"],
        "entity_f1": entity_f1
    }

# Initialize trainer (เหมือนเดิม)
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# Train the model (เหมือนเดิม)
trainer.train()

# Save the final model (เหมือนเดิม)
final_model_path = os.path.join(OUTPUT_DIR, "final_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"Training completed! Model saved to {final_model_path}")

# Evaluate and print results (เหมือนเดิม)
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

# Test the model with validation dataset (ไม่ใส่ Confusion Matrix ซ้ำ)
def test_model(trainer, dataset, num_examples=7):
    print("\n=== Testing Model ===")
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    test_indices = np.random.choice(len(dataset), num_examples, replace=False)
    
    for idx in test_indices:
        example = dataset[idx]
        tokens = dataset.data[idx]["tokens"]
        true_labels = dataset.data[idx]["ner_tags"]

        inputs = {key: val.unsqueeze(0).to(device) for key, val in example.items() if key != "labels"}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=2).squeeze(0).cpu().numpy()

        word_ids = tokenizer(tokens, is_split_into_words=True).word_ids(batch_index=0)
        aligned_predictions = []
        prev_word_idx = None
        for i, word_idx in enumerate(word_ids):
            if word_idx is None:
                continue
            elif word_idx != prev_word_idx:
                aligned_predictions.append(predictions[i])
            prev_word_idx = word_idx

        aligned_predictions = aligned_predictions[:len(true_labels)]
        true_labels_str = [id2label.get(label, "INVALID") for label in true_labels]
        pred_labels_str = [id2label.get(pred, "INVALID") for pred in aligned_predictions]
        matching = ["✅" if t == p else "❌" for t, p in zip(true_labels, aligned_predictions)]

        table_data = {
            "Token": tokens,
            "True Label": true_labels_str,
            "Predicted Label": pred_labels_str,
            "Match": matching
        }
        df = pd.DataFrame(table_data)
        
        print(f"\nExample {idx}:")
        print(f"Sentence: {' '.join(tokens)}")
        print(df.to_string(index=False))

# Run the test
test_model(trainer, validate_dataset, num_examples=7)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...


=== Confusion Matrix (Evaluation) ===
                        O  B-CLOUDPLATFORM  I-CLOUDPLATFORM  B-PROGRAMMINGLANG  I-PROGRAMMINGLANG  B-FRAMEWORK_LIB  I-FRAMEWORK_LIB  B-WEBFRAMEWORK_TECH  I-WEBFRAMEWORK_TECH  B-DATABASE  I-DATABASE  B-EMBEDDEDTECH
O                    5470               92               13                103                  5              122               18                   46                    0         127           4               0
B-CLOUDPLATFORM         2               18                0                  0                  0                0                0                    0                    0           0           0               0
I-CLOUDPLATFORM         0                0                1                  0                  0                0                0                    0                    0           0           0               0
B-PROGRAMMINGLANG       6                0                0                115                  0        

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


=== Confusion Matrix (Evaluation) ===
                        O  B-CLOUDPLATFORM  I-CLOUDPLATFORM  B-PROGRAMMINGLANG  I-PROGRAMMINGLANG  B-FRAMEWORK_LIB  I-FRAMEWORK_LIB  B-WEBFRAMEWORK_TECH  I-WEBFRAMEWORK_TECH  B-DATABASE  I-DATABASE  B-EMBEDDEDTECH
O                    5470               92               13                103                  5              122               18                   46                    0         127           4               0
B-CLOUDPLATFORM         2               18                0                  0                  0                0                0                    0                    0           0           0               0
I-CLOUDPLATFORM         0                0                1                  0                  0                0                0                    0                    0           0           0               0
B-PROGRAMMINGLANG       6                0                0                115                  0        

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


=== Confusion Matrix (Evaluation) ===
                        O  B-CLOUDPLATFORM  I-CLOUDPLATFORM  B-PROGRAMMINGLANG  I-PROGRAMMINGLANG  B-FRAMEWORK_LIB  I-FRAMEWORK_LIB  B-WEBFRAMEWORK_TECH  I-WEBFRAMEWORK_TECH  B-DATABASE  I-DATABASE  B-EMBEDDEDTECH
O                    5477               93               13                 99                  5              124               17                   41                    0         127           4               0
B-CLOUDPLATFORM         2               18                0                  0                  0                0                0                    0                    0           0           0               0
I-CLOUDPLATFORM         0                0                1                  0                  0                0                0                    0                    0           0           0               0
B-PROGRAMMINGLANG       6                0                0                115                  0        